In [1]:
from scipy.stats import ttest_rel, ttest_ind, wilcoxon

In [2]:
# Sample data - RMSE scores for original and replica models
pls_original = [4.33, 0.94, 2.85, 2.01, 1.06, 2.65, 0.62, 0.72]
pls_replica = [5.81, 0.47, 1.94, 4.35, 1.17, 1.43, 0.66, 0.72]

ica_original = [8.31, 1.44, 4.77, 5.17, 4.08, 3.07, 2.29, 0.98]
ica_replica = [10.68, 0.63, 5.55, 8.30, 2.90, 3.52, 1.72, 1.37]

moc_original = [5.30, 1.03, 3.47, 2.31, 2.21, 2.72, 0.62, 0.82]
moc_replica = [7.29, 0.49, 2.39, 5.21, 1.67, 1.81, 1.10, 1.09]


In [3]:
def perform_student_t_test(original, replica):
	t_stat, p_value = ttest_rel(original, replica)
	return t_stat, p_value

print(f"PLS Student's t-test: {perform_student_t_test(pls_original, pls_replica)}")
print(f"ICA Student's t-test: {perform_student_t_test(ica_original, ica_replica)}")
print(f"MOC Student's t-test: {perform_student_t_test(moc_original, moc_replica)}")


PLS Student's t-test: (-0.4056067802087878, 0.6971426466917101)
ICA Student's t-test: (-1.0613757495947185, 0.32376000700705115)
MOC Student's t-test: (-0.6329536381030276, 0.5468768019604969)


In [4]:
def perform_welchs_test(original, replica):
    t_stat, p_value = ttest_ind(original, replica, equal_var=False)
    return t_stat, p_value

print(f"PLS1-SM Welch's: {perform_welchs_test(pls_original, pls_replica)}")
print(f"ICA Welch's: {perform_welchs_test(ica_original, ica_replica)}")
print(f"MOC Welch's: {perform_welchs_test(moc_original, moc_replica)}")

PLS1-SM Welch's: (-0.20564772648848437, 0.8404483397001741)
ICA Welch's: (-0.3754140305999682, 0.7138111860718925)
MOC Welch's: (-0.3198421165467493, 0.7545270443016014)


In [5]:
def perform_wilcoxon_test(original, replica):
    w_stat, p_value = wilcoxon(original, replica)
    return w_stat, p_value

print(f"PLS1-SM Wilcoxon: {perform_wilcoxon_test(pls_original, pls_replica)}")
print(f"ICA Wilcoxon: {perform_wilcoxon_test(ica_original, ica_replica)}")
print(f"MOC Wilcoxon: {perform_wilcoxon_test(moc_original, moc_replica)}")

PLS1-SM Wilcoxon: (12.0, 0.7353166906373405)
ICA Wilcoxon: (14.0, 0.640625)
MOC Wilcoxon: (18.0, 1.0)


c:\Users\Patrick\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Patrick\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_morestats.py:3159: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
